<a href="https://colab.research.google.com/github/m-Ash1/sentiment-analysis/blob/master/Multicontest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Data preprocessing


In [ ]:
nlp = spacy.load('en_core_web_sm')

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [ ]:
def preprocessing(data):
  data['new_tweets'] = lemmatization(data['tweet'])
  tweets = []
  for index in range(0,len(data)):
    tweet = re.sub("(http|https)\S*", "", data.at[index,"new_tweets"])
    tweet = re.sub('[^A-Za-z]+', ' ', tweet)
    tweet = re.sub("[\s]+", " ", tweet)
    tweet = tweet.lower()
    tweets.append(tweet)

  data['new_tweets'] = tweets
  return data

In [ ]:
preprocessing(train)
preprocessing(test)

,id,tweet,new_tweets
0,7921,I hate the new #iphone upgrade. Won't let me d...,i hate the new iphone upgrade will not let i d...
1,7922,currently shitting my fucking pants. #apple #i...,currently shit my fucking pant apple imac cash...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",i would like to puts some cd roms on my ipad b...
3,7924,My ipod is officially dead. I lost all my pict...,my ipod be officially dead i lose all my pictu...
4,7925,Been fighting iTunes all night! I only want th...,be fight itunes all night i only want the musi...
...,...,...,...
1948,9869,"#SamsungGalaxyNote7 Explodes, Burns 6-Year-Old...",samsunggalaxynote explodes burns year old tha...
1949,9870,Now Available - Hoodie. Check it out here - ht...,now available hoodie check it out here iphone ...
1950,9871,There goes a crack right across the screen. If...,there go a crack right across the screen if yo...
1951,9872,@codeofinterest as i said #Adobe big time we m...,codeofinter as i say adobe big time we may we...


## Feature Extraction

In [ ]:
elmo = hub.load("https://tfhub.dev/google/elmo/2").signatures["default"]

In [ ]:
def elmo_vectors(x):
  embeddings = elmo(tf.constant(x))["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
elmo_train = [elmo_vectors(x['new_tweets']) for x in list_train]

In [ ]:
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]
elmo_test = [elmo_vectors(x['new_tweets']) for x in list_test]

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression()
lreg.fit(elmo_train_new, train['label'])

In [ ]:
preds_test = lreg.predict(elmo_test_new)

# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)